In [1]:
import glob
import os
import numpy as np
from random import shuffle
from nltk.tokenize import TreebankWordTokenizer
from nlpia.loaders import get_data

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN

C:\Users\voldo\AppData\Local\Programs\Python\Python39\lib\site-packages\pugnlp\constants.py:136: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  [datetime.datetime, pd.datetime, pd.Timestamp])
C:\Users\voldo\AppData\Local\Programs\Python\Python39\lib\site-packages\pugnlp\constants.py:158: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  MIN_TIMESTAMP = pd.Timestamp(pd.datetime(1677, 9, 22, 0, 12, 44), tz='utc')
C:\Users\voldo\AppData\Local\Programs\Python\Python39\lib\site-packages\pugnlp\tutil.py:100: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  np = pd.np
C:\Users\voldo\AppData\Local\Programs\Python\Python39\lib\site-packages\pugnlp\util.py:80: FutureWarning: The pandas.np module is deprecated and wil

In [2]:
word_vectors = get_data('wv', limit=200_000)

INFO:nlpia.loaders:Downloading wv
INFO:nlpia.web:URL too short: wv
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.300d\.zip$, string=googlenews-vectors-negative300.bin.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.27b\.zip$, string=googlenews-vectors-negative300.bin.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.42b\.zip$, string=googlenews-vectors-negative300.bin.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.6b\.zip$, string=googlenews-vectors-negative300.bin.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.bin\.gz$, string=googlenews-vectors-negative300.bin.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.tgz$, string=googlenews-vectors-negative300.bin.gz
INFO:nlpia.loaders:expanded+normalized file path: C:\Users\voldo\AppData\Local\Programs\Python\Python39\lib\site-packages\nlpia\bigdata\googlenews-vectors-negative300.bin.gz
INFO:nlpia.loaders:requesting URL: https://www.drop

In [3]:
def preprocess_data(filepath_):
    positive_path_ = os.path.join(filepath_, 'pos')
    negative_path_ = os.path.join(filepath_, 'neg')
    pos_label_ = 1
    neg_label_ = 0
    dataset_ = []

    for filename_ in glob.glob(os.path.join(positive_path_, '*.txt')):
        try:
            with open(filename_, 'r') as f_:
                dataset_.append((pos_label_, f_.read()))
        except UnicodeDecodeError:
            pass

    for filename_ in glob.glob(os.path.join(negative_path_, '*.txt')):
        try:
            with open(filename_, 'r') as f_:
                dataset_.append((neg_label_, f_.read()))
        except UnicodeDecodeError:
            pass

    shuffle(dataset_)
    return dataset_

In [4]:
def tokenize_and_vectorize(dataset_):
    tokenizer_ = TreebankWordTokenizer()
    vectorized_data_ = []
    expected_ = []
    for sample_ in dataset_:
        tokens_ = tokenizer_.tokenize(sample_[1])
        sample_vecs_ = []
        for token_ in tokens_:
            try:
                sample_vecs_.append(word_vectors[token_])
            except KeyError:
                pass
        vectorized_data_.append(sample_vecs_)
        expected_.append(sample_[0])
    return vectorized_data_, expected_

In [10]:
def pad_trunc(data_, maxlen_):
    new_data_ = []
    zero_vector_ = [0.0 for _ in range(len(data_[0][0]))]
    for sample_ in data_:
        if len(sample_) > maxlen_:
            temp_ = sample_[:maxlen_]
        elif len(sample_) < maxlen_:
            temp_ = sample_
            additional_elems_ = maxlen_ - len(sample_)
            for _ in range(additional_elems_):
                temp_.append(zero_vector_)
        else:
            temp_ = sample_
        new_data_.append(temp_)
    return new_data_

In [5]:
dataset = preprocess_data("../src/part_6/stanford_sent_analysis_dataset/aclImdb/train")
print("Because I don't have enough memory: I'll crop the dataset:")
dataset = dataset[:5000]
print('Len new dataset:', len(dataset))
vectorized_data, expected = tokenize_and_vectorize(dataset)
del dataset
del word_vectors

Because I don't have enough memory: I'll crop the dataset:
Len new dataset: 5000


In [6]:
split_point = int(len(vectorized_data) * .8)
x_train = vectorized_data[:split_point]
y_train = expected[:split_point]

x_test = vectorized_data[split_point:]
y_test = expected[split_point:]
del expected

In [7]:
print("\n\nTotal dataset parameters:")
print("Dataset size:", len(vectorized_data))
print("Training set size:", len(x_train))
print("Test set size:", len(x_test), "\n\n")
del vectorized_data



Total dataset parameters:
Dataset size: 5000
Training set size: 4000
Test set size: 1000 




In [8]:
# RNN params:
maxlen = 400
batch_size = 32
embedding_dims = 300
epochs = 5
num_neurons = 25

In [11]:
# pad + trunkate the data
x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)

x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)

In [12]:
# build the RNN
model = Sequential()
model.add(SimpleRNN(units=num_neurons,
                    return_sequences=True,
                    input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print("\n\nBuilding the RNN model:")
model.summary()



Building the RNN model:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 400, 25)           8150      
                                                                 
 dropout (Dropout)           (None, 400, 25)           0         
                                                                 
 flatten (Flatten)           (None, 10000)             0         
                                                                 
 dense (Dense)               (None, 1)                 10001     
                                                                 
Total params: 18,151
Trainable params: 18,151
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Epoch 1/5
125/125 [==============================] - 28s 197ms/step - loss: 0.7008 - accuracy: 0.5540 - val_loss: 0.7061 - val_accuracy: 0.5850
Epoch 2/5
125/125 [==============================] - 23s 185ms/step - loss: 0.4996 - accuracy: 0.7580 - val_loss: 0.5565 - val_accuracy: 0.7160
Epoch 3/5
125/125 [==============================] - 23s 185ms/step - loss: 0.4040 - accuracy: 0.8108 - val_loss: 0.5409 - val_accuracy: 0.7410
Epoch 4/5
125/125 [==============================] - 24s 193ms/step - loss: 0.3397 - accuracy: 0.8593 - val_loss: 0.5454 - val_accuracy: 0.7500
Epoch 5/5
125/125 [==============================] - 26s 205ms/step - loss: 0.2800 - accuracy: 0.8823 - val_loss: 0.6071 - val_accuracy: 0.7440


In [14]:
# save model to file:
model_structure = model.to_json()
with open('./rnn_model.json', 'w') as json_file:
    json_file.write(model_structure)
model.save_weights('./rnn_weights.h5')